In [2]:
# Imports

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
from datetime import datetime, timedelta

In [4]:
# Get the current date
today = datetime.today().date()

# Get the date 30 days from today
today_plus_30 = today + timedelta(days=30) #TODO configurable to days in advance, default 30

#today.year, today.month, today.day

(2024, 11, 30)

In [5]:
# https://www.eventim.de/events/konzerte-1/?zipcode=24534&distance=100&shownonbookable=true&sort=DateAsc&dateFrom=2024-11-30&dateTo=2025-01-31 #erstmal nur Konzerte, ist ok oder?

url = f"https://www.eventim.de/events/konzerte-1/?zipcode=24534&distance=100&shownonbookable=true&sort=DateAsc&dateFrom={today.year}-{today.month}-{today.day}&dateTo={today_plus_30.year}-{today_plus_30.month}-{today_plus_30.day}"
url

'https://www.eventim.de/events/konzerte-1/?zipcode=24534&distance=100&shownonbookable=true&sort=DateAsc&dateFrom=2024-11-30&dateTo=2024-12-30'

In [6]:
driver = webdriver.Firefox()

In [29]:
driver.get(url)

In [8]:
# cookie rejection
try:
    element = driver.find_element(By.ID, "cmpwelcomebtnno")
    element.click()
    print("Element clicked successfully.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Element clicked successfully.


In [30]:
wait = WebDriverWait(driver, 10)
elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "product-group-item")))
elements

[<selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="18143b4c-f742-43ba-9c12-513d6698d329")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="bdcbba84-4c86-4f91-9da7-bd41a9624d90")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="8bb5203a-9823-43a6-b4f4-68bc40326b7c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="5338f7af-c935-4928-931e-6f8730ce40b3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="4b72d058-fef2-42fd-a5f2-bb9e07d645ee")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element="48304d0d-578c-4a2a-96cd-dc437c0fdc4e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e9d08c5-4674-4d56-93a4-c601adb31942", element

In [26]:
#elements[2].find_element(By.ID, "listing-headline-2")

In [32]:
extracted_info = []   

# Iterate over each element and extract the required information
for element in elements:
    #print(element)
    try:
        # Extract the title
        title_element = element.find_element(By.CSS_SELECTOR, '[id^="listing-headline"]')#(By.ID, "listing-headline-0")
        title = title_element.text
        #print(title)

        # Extract the location/date/time
        location_date_time_element = element.find_element(By.CSS_SELECTOR, ".text-overflow-ellipsis.u-text-color.theme-text-color")
        location_date_time = location_date_time_element.text

        # Extract the description (if it exists)
        try:
            description_element = element.find_element(By.CSS_SELECTOR, ".listing-description.theme-text-color.text-overflow-ellipsis.hidden-xs")
            description = description_element.text
        except Exception as e:
            description = None

        # Extract the source (href)
        source_element = element.find_element(By.CSS_SELECTOR, "a.btn.btn-sm.btn-block.btn-primary")
        source = source_element.get_attribute("href")

        # Append the extracted information to the list
        extracted_info.append({
            "Subject": title,
            "Location_Date_Time": location_date_time,
            "Description": description,
            "Source": source,
            "Category": "Konzert",
            "Music_label": "music"
        })

    except Exception as e:
        continue
        #print(f"Required element not found: {e}")


In [33]:
extracted_info

[{'Subject': 'Plage Noire 2024',
  'Location_Date_Time': 'Weissenhäuser Strand, Fr. 29.11.2024 ─ Sa. 30.11.2024',
  'Description': "Mit Eisbrecher, Lord Of The Lost, Subway To Sally, Project Pitchfork, Diary Of Dreams, Front Line Assembly, L'Âme Immortelle, Hocico u .v. m.",
  'Source': 'https://www.eventim.de/artist/plage-noire/plage-noire-2024-3389094/',
  'Category': 'Konzert',
  'Music_label': 'music'},
 {'Subject': 'Rhonda - This Is The End Tour 2024',
  'Location_Date_Time': 'Hamburg, Sa. 30.11.2024, 18:30',
  'Description': '',
  'Source': 'https://www.eventim.de/artist/rhonda/',
  'Category': 'Konzert',
  'Music_label': 'music'},
 {'Subject': 'DEAFCON5',
  'Location_Date_Time': 'Hamburg, Sa. 30.11.2024, 19:00',
  'Description': '',
  'Source': 'https://www.eventim.de/artist/deafcon5/',
  'Category': 'Konzert',
  'Music_label': 'music'},
 {'Subject': 'Kanii - #Blue Tour',
  'Location_Date_Time': 'Hamburg, Sa. 30.11.2024, 19:00',
  'Description': '',
  'Source': 'https://www.even

In [45]:
df_raw = pd.DataFrame(extracted_info)
df_raw

,Subject,Location_Date_Time,Description,Source,Category,Music_label
0,Plage Noire 2024,"Weissenhäuser Strand, Fr. 29.11.2024 ─ Sa. 30....","Mit Eisbrecher, Lord Of The Lost, Subway To Sa...",https://www.eventim.de/artist/plage-noire/plag...,Konzert,music
1,Rhonda - This Is The End Tour 2024,"Hamburg, Sa. 30.11.2024, 18:30",,https://www.eventim.de/artist/rhonda/,Konzert,music
2,DEAFCON5,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/deafcon5/,Konzert,music
3,Kanii - #Blue Tour,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/kanii/,Konzert,music
4,FEUERENGEL,"Lübeck, Sa. 30.11.2024, 20:00",Europas beste RAMMSTEIN-Tribute-Show live!,https://www.eventim.de/artist/feuerengel/feuer...,Konzert,music
5,Nazareth,"Flensburg, Sa. 30.11.2024, 20:00",,https://www.eventim.de/artist/nazareth/,Konzert,music
6,Ilse DeLange - Tour Germany 2024,"Hamburg, Sa. 30.11.2024, 20:00",2024 wird man Ilse DeLange bei ausgewählten Ko...,https://www.eventim.de/artist/ilse-delange/ils...,Konzert,music
7,Rocket Men - Lost in Space,"Hamburg, Sa. 30.11.2024, 20:00",,https://www.eventim.de/eventseries/rocket-men-...,Konzert,music
8,"Mary Roos & Wolfgang Trepper: Mehr Nutten, meh...",GP JOULE Arena,Die Fortsetzung des Erfolgsprogramms der Theat...,https://www.eventim.de/artist/mary-roos/,Konzert,music
9,The Beatles Today: Now And Then,"Lübeck, Sa. 30.11.2024, 20:00",„THE BEATLES TODAY“ – eine Hommage an die größ...,https://www.eventim.de/artist/the-beatles-today/,Konzert,music


In [43]:
try:
    #pagination_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "pagination-item .btn.btn-lg.btn-square")))
    pagination_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "pagination-item a[data-qa='nextPage']")))
    pagination_element.click()
    time.sleep(5)
    #print("Pagination element clicked successfully.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    #break

An unexpected error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



In [44]:
driver.close()

In [46]:
def split_location_date_time(value):
    parts = value.split(',')
    if len(parts) == 3:
        return parts[0].strip(), parts[1].strip(), parts[2].strip()
    elif len(parts) == 2:
        return parts[0].strip(), parts[1].strip(), "N/A"
    else:
        return value.strip(), "N/A", "N/A"

In [47]:
df_raw[['Location', 'Date', 'Time']] = df_raw['Location_Date_Time'].apply(lambda x: pd.Series(split_location_date_time(x)))

In [49]:
df_raw

,Subject,Location_Date_Time,Description,Source,Category,Music_label,Location,Date,Time
0,Plage Noire 2024,"Weissenhäuser Strand, Fr. 29.11.2024 ─ Sa. 30....","Mit Eisbrecher, Lord Of The Lost, Subway To Sa...",https://www.eventim.de/artist/plage-noire/plag...,Konzert,music,Weissenhäuser Strand,Fr. 29.11.2024 ─ Sa. 30.11.2024,N/A
1,Rhonda - This Is The End Tour 2024,"Hamburg, Sa. 30.11.2024, 18:30",,https://www.eventim.de/artist/rhonda/,Konzert,music,Hamburg,Sa. 30.11.2024,18:30
2,DEAFCON5,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/deafcon5/,Konzert,music,Hamburg,Sa. 30.11.2024,19:00
3,Kanii - #Blue Tour,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/kanii/,Konzert,music,Hamburg,Sa. 30.11.2024,19:00
4,FEUERENGEL,"Lübeck, Sa. 30.11.2024, 20:00",Europas beste RAMMSTEIN-Tribute-Show live!,https://www.eventim.de/artist/feuerengel/feuer...,Konzert,music,Lübeck,Sa. 30.11.2024,20:00
5,Nazareth,"Flensburg, Sa. 30.11.2024, 20:00",,https://www.eventim.de/artist/nazareth/,Konzert,music,Flensburg,Sa. 30.11.2024,20:00
6,Ilse DeLange - Tour Germany 2024,"Hamburg, Sa. 30.11.2024, 20:00",2024 wird man Ilse DeLange bei ausgewählten Ko...,https://www.eventim.de/artist/ilse-delange/ils...,Konzert,music,Hamburg,Sa. 30.11.2024,20:00
7,Rocket Men - Lost in Space,"Hamburg, Sa. 30.11.2024, 20:00",,https://www.eventim.de/eventseries/rocket-men-...,Konzert,music,Hamburg,Sa. 30.11.2024,20:00
8,"Mary Roos & Wolfgang Trepper: Mehr Nutten, meh...",GP JOULE Arena,Die Fortsetzung des Erfolgsprogramms der Theat...,https://www.eventim.de/artist/mary-roos/,Konzert,music,GP JOULE Arena,N/A,N/A
9,The Beatles Today: Now And Then,"Lübeck, Sa. 30.11.2024, 20:00",„THE BEATLES TODAY“ – eine Hommage an die größ...,https://www.eventim.de/artist/the-beatles-today/,Konzert,music,Lübeck,Sa. 30.11.2024,20:00


# Function

In [ ]:
def scrape_eventim(days_in_advance=30):

    # Get the current date
    today = datetime.today().date()
    # Get the date x days from today
    today_plus_x = today + timedelta(days=days_in_advance)

    #define url
    url = f"https://www.eventim.de/events/konzerte-1/?zipcode=24534&distance=100&shownonbookable=true&sort=DateAsc&dateFrom={today.year}-{today.month}-{today.day}&dateTo={today_plus_x.year}-{today_plus_x.month}-{today_plus_x.day}"

    #prepare scraping
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(5)
    wait = WebDriverWait(driver, 10)

    # cookie rejection
    try:
        element = driver.find_element(By.ID, "cmpwelcomebtnno")
        element.click()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    extracted_info = [] 

    while True:
        elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "product-group-item")))

        # Iterate over each element and extract the required information
        for element in elements:
            try:
                # Extract the title
                title_element = element.find_element(By.CSS_SELECTOR, '[id^="listing-headline"]')#(By.ID, "listing-headline-0")
                title = title_element.text

                # Extract the location/date/time
                location_date_time_element = element.find_element(By.CSS_SELECTOR, ".text-overflow-ellipsis.u-text-color.theme-text-color")
                location_date_time = location_date_time_element.text

                # Extract the description (if it exists)
                try:
                    description_element = element.find_element(By.CSS_SELECTOR, ".listing-description.theme-text-color.text-overflow-ellipsis.hidden-xs")
                    description = description_element.text
                except Exception as e:
                    description = None

                # Extract the source (href)
                source_element = element.find_element(By.CSS_SELECTOR, "a.btn.btn-sm.btn-block.btn-primary")
                source = source_element.get_attribute("href")

                # Append the extracted information to the list
                extracted_info.append({
                    "Subject": title,
                    "Location_Date_Time": location_date_time,
                    "Description": description,
                    "Source": source,
                    "Category": "Konzert",
                    "Music_label": "music"
                })

            except Exception as e:
                continue
        
        try:
            pagination_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "pagination-item a[data-qa='nextPage']")))
            pagination_element.click()
            time.sleep(5)
        except Exception as e:
            print("No more pages")
            break

    df_raw = pd.DataFrame(extracted_info)
    driver.close()
    return df_raw


In [51]:
df_raw = scrape_eventim()
df_raw

No more pages


,Subject,Location_Date_Time,Description,Source,Category,Music_label
0,Plage Noire 2024,"Weissenhäuser Strand, Fr. 29.11.2024 ─ Sa. 30....","Mit Eisbrecher, Lord Of The Lost, Subway To Sa...",https://www.eventim.de/artist/plage-noire/plag...,Konzert,music
1,Rhonda - This Is The End Tour 2024,"Hamburg, Sa. 30.11.2024, 18:30",,https://www.eventim.de/artist/rhonda/,Konzert,music
2,DEAFCON5,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/deafcon5/,Konzert,music
3,Kanii - #Blue Tour,"Hamburg, Sa. 30.11.2024, 19:00",,https://www.eventim.de/artist/kanii/,Konzert,music
4,Ilse DeLange - Tour Germany 2024,"Hamburg, Sa. 30.11.2024, 20:00",2024 wird man Ilse DeLange bei ausgewählten Ko...,https://www.eventim.de/artist/ilse-delange/ils...,Konzert,music
...,...,...,...,...,...,...
252,Prinz Pi - Kompass ohne Norden Tour - 10 Jahre,"Hamburg, So. 29.12.2024, 20:00",Prinz Pi feiert 2024 das 10-jährige Jubiläum v...,https://www.eventim.de/artist/prinz-pi/prinz-p...,Konzert,music
253,The Chambers - Die Virtuosen aus Köln,"Hemmor, Mo. 30.12.2024, 19:00",,https://www.eventim.de/artist/the-chambers/,Konzert,music
254,Ski's Country Trash,"Hamburg, Mo. 30.12.2024, 20:00","Gambrinus Klingenthal"" präsentiert: SKI'S COUN...",https://www.eventim.de/artist/skis-country-trash/,Konzert,music
255,Die Happy - Last Christmas Tour 2024,"Hamburg, Mo. 30.12.2024, 20:00",,https://www.eventim.de/artist/die-happy/,Konzert,music


In [52]:
save_df = df_raw.copy()

In [68]:
df_raw.head()

,Subject,Description,Category,Music_label,Location,Start_date,Start_time,End_time,End_date
0,Plage Noire 2024,"Mit Eisbrecher, Lord Of The Lost, Subway To Sa...",Konzert,music,Weissenhäuser Strand,30.11.2024,N/A,N/A,30.11.2024
1,Rhonda - This Is The End Tour 2024,https://www.eventim.de/artist/rhonda/,Konzert,music,Hamburg,30.11.2024,18:30,N/A,30.11.2024
2,DEAFCON5,https://www.eventim.de/artist/deafcon5/,Konzert,music,Hamburg,30.11.2024,19:00,N/A,30.11.2024
3,Kanii - #Blue Tour,https://www.eventim.de/artist/kanii/,Konzert,music,Hamburg,30.11.2024,19:00,N/A,30.11.2024
4,Ilse DeLange - Tour Germany 2024,2024 wird man Ilse DeLange bei ausgewählten Ko...,Konzert,music,Hamburg,30.11.2024,20:00,N/A,30.11.2024


In [54]:
def split_location_date_time(value):
    parts = value.split(',')
    if len(parts) == 3:
        return parts[0].strip(), parts[1].strip(), parts[2].strip()
    elif len(parts) == 2:
        return parts[0].strip(), parts[1].strip(), "N/A"
    else:
        return value.strip(), "N/A", "N/A"

In [55]:
df_raw[['Location', 'Date', 'Time']] = df_raw['Location_Date_Time'].apply(lambda x: pd.Series(split_location_date_time(x)))

In [57]:
df_raw.drop(columns=['Location_Date_Time'], inplace=True)

In [59]:
df_raw["Description"] = df_raw["Description"] + " " + df_raw["Source"]

In [61]:
df_raw.drop(columns=['Source'], inplace=True)

In [63]:
df_raw.rename(columns={'Time': 'Start_time'}, inplace=True)
df_raw["End_time"] = "N/A"

In [65]:
df_raw['Date'] = df_raw['Date'].str[-10:]

In [67]:
df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)
df_raw["End_date"] = df_raw["Start_date"]

In [ ]:
# Preprocessing function

def preprocess_eventim(df_raw):

    df_raw[['Location', 'Date', 'Time']] = df_raw['Location_Date_Time'].apply(lambda x: pd.Series(split_location_date_time(x)))
    df_raw.drop(columns=['Location_Date_Time'], inplace=True)

    df_raw["Description"] = df_raw["Description"] + " " + df_raw["Source"]
    df_raw.drop(columns=['Source'], inplace=True)

    df_raw.rename(columns={'Time': 'Start_time'}, inplace=True)
    df_raw["End_time"] = "N/A"

    df_raw['Date'] = df_raw['Date'].str[-10:]
    df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)
    df_raw["End_date"] = df_raw["Start_date"]

    df_raw["City"] = df_raw["Location"]

    df_prep = df_raw[['Subject','Start_date', 'End_date', 'Start_time', 'End_time', 'Location', 'City', 'Description', 'Category', 'Music_label']]
    return df_prep

In [74]:
df_prep = preprocess_eventim(save_df)
df_prep

,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,Plage Noire 2024,30.11.2024,30.11.2024,N/A,N/A,Weissenhäuser Strand,Weissenhäuser Strand,"Mit Eisbrecher, Lord Of The Lost, Subway To Sa...",Konzert,music
1,Rhonda - This Is The End Tour 2024,30.11.2024,30.11.2024,18:30,N/A,Hamburg,Hamburg,https://www.eventim.de/artist/rhonda/,Konzert,music
2,DEAFCON5,30.11.2024,30.11.2024,19:00,N/A,Hamburg,Hamburg,https://www.eventim.de/artist/deafcon5/,Konzert,music
3,Kanii - #Blue Tour,30.11.2024,30.11.2024,19:00,N/A,Hamburg,Hamburg,https://www.eventim.de/artist/kanii/,Konzert,music
4,Ilse DeLange - Tour Germany 2024,30.11.2024,30.11.2024,20:00,N/A,Hamburg,Hamburg,2024 wird man Ilse DeLange bei ausgewählten Ko...,Konzert,music
...,...,...,...,...,...,...,...,...,...,...
252,Prinz Pi - Kompass ohne Norden Tour - 10 Jahre,29.12.2024,29.12.2024,20:00,N/A,Hamburg,Hamburg,Prinz Pi feiert 2024 das 10-jährige Jubiläum v...,Konzert,music
253,The Chambers - Die Virtuosen aus Köln,30.12.2024,30.12.2024,19:00,N/A,Hemmor,Hemmor,https://www.eventim.de/artist/the-chambers/,Konzert,music
254,Ski's Country Trash,30.12.2024,30.12.2024,20:00,N/A,Hamburg,Hamburg,"Gambrinus Klingenthal"" präsentiert: SKI'S COUN...",Konzert,music
255,Die Happy - Last Christmas Tour 2024,30.12.2024,30.12.2024,20:00,N/A,Hamburg,Hamburg,https://www.eventim.de/artist/die-happy/,Konzert,music
